In [ ]:
import mysql.connector

# Replace these with your own database connection details
db_config = {
    "host": "",
    "user": "your_username",
    "password": "your_password",
    "database": "your_database"
}

# Establish a database connection
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

In [1]:
import cv2
import pytesseract
import re

# Load the invoice image
img_path = "C:\\Users\\asus\\Downloads\\s5.jpg"
img = cv2.imread(img_path)

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Use pytesseract to extract text
text = pytesseract.image_to_string(gray)

# Define a dictionary of keywords and corresponding information keys
keywords = {
    "Invoice Date": "Invoice Date",
}

# Define regular expression patterns
invoice_number_patterns = [
    r'Invoice\s*No[.\s]*:\s*([A-Za-z0-9]+)',       # Existing pattern
    r'Invoice\s*No[.\s]*\s*:\s*([0-9A-Za-z]+)\s*'  # New format: Invoice No.
]

# Search for the invoice number using the regular expressions
invoice_number = "Not found"
for pattern in invoice_number_patterns:
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        invoice_number = match.group(1)
        break

# Print the extracted invoice number
print(f"Invoice Number: {invoice_number}")

gst_number_pattern = r'GSTIN\s*:\s*([A-Za-z0-9]+)'
gstin_uin_pattern = r'GSTIN/ UIN\s*:\s*([A-Za-z0-9]+)'

# Search for the GSTIN/UIN using the regular expressions
gst_number_match = re.search(gst_number_pattern, text, re.IGNORECASE)
gstin_uin_match = re.search(gstin_uin_pattern, text, re.IGNORECASE)

# Extract the GSTIN/UIN if a match is found
gst_number = "Not found"
gstin_uin = "Not found"

if gst_number_match:
    gst_number = gst_number_match.group(1)

if gstin_uin_match:
    gstin_uin = gstin_uin_match.group(1)

# Print the extracted GSTIN and GSTIN/UIN
print(f"GSTIN: {gst_number}")
print(f"GSTIN/UIN: {gstin_uin}")

# Define possible date formats
date_formats = [
    r'\d{1,2}/\d{1,2}/\d{2,4}',  # e.g., 08/23/2023
    r'\d{1,2}-\d{1,2}-\d{2,4}',  # e.g., 08-23-2023
    r'\d{1,2}\s[a-zA-Z]{3,9}\s\d{2,4}',  # e.g., 23 Aug 2023
    r'\d{1,2}\s[a-zA-Z]{3,9},\s\d{2,4}',  # e.g., 23 Aug, 2023
    r'[a-zA-Z]{3,9}\s\d{1,2},\s\d{2,4}',  # e.g., Aug 23, 2023
    r'[a-zA-Z]{3,9}\s\d{1,2}\s\d{2,4}',  # e.g., Aug 23 2023
    r'\d{1,2}\s[a-zA-Z]{3,9}\s\'\d{2}',  # e.g., 23 Aug '23
    r'\d{1,2}\s[a-zA-Z]{3,9}\s\d{4}',  # e.g., 23 Aug 2023
    r'\d{1,2}-[a-zA-Z]{3}-\d{2,4}',  # e.g., 23-Jul-2023
    # Add more date formats as needed
]

state_code_pattern = r'State\s*Code\s*:\s*([A-Za-z0-9]+)'

# Search for the state code using the regular expression
state_code_match = re.search(state_code_pattern, text, re.IGNORECASE)

# Extract the state code if a match is found
if state_code_match:
    state_code = state_code_match.group(1)
else:
    state_code = "Not found"

# Print the extracted state code
print(f"State Code: {state_code}")

# Extract information using keywords and date formats
extracted_info = {}

for keyword, info_key in keywords.items():
    start_idx = text.find(keyword)
    if start_idx != -1:
        start_idx += len(keyword) + 1  # Move past the colon
        end_idx = text.find("\n", start_idx)
        value = text[start_idx:end_idx].strip()

        # Try to extract date using different formats
        date_value = "Not found"
        for date_format in date_formats:
            match = re.search(date_format, value)
            if match:
                date_value = match.group()
                break

        extracted_info[info_key] = date_value
    else:
        extracted_info[info_key] = "Not found"

# Print the extracted information
for info_key, value in extracted_info.items():
    print(f"{info_key}: {value}")
    
    
 #Define the region coordinates for "Billed to" and "Supplied to" boxes
billed_to_box = (80, 355, 601, 514)  # Adjust these coordinates as needed
supplied_to_box = (603, 355, 1129, 518)  # Adjust these coordinates as needed
total_amount_box = (777,767,1130,795)
total_tax_box = (779,738,1129,765)

# Extract text from the "Billed to" and "Supplied to" boxes
billed_to_text = pytesseract.image_to_string(img[billed_to_box[1]:billed_to_box[3], billed_to_box[0]:billed_to_box[2]])
supplied_to_text = pytesseract.image_to_string(img[supplied_to_box[1]:supplied_to_box[3], supplied_to_box[0]:supplied_to_box[2]])
total_tax_text = pytesseract.image_to_string(img[total_tax_box[1]:total_tax_box[3], total_tax_box[0]:total_tax_box[2]])
total_amount_text = pytesseract.image_to_string(img[total_amount_box[1]:total_amount_box[3], total_amount_box[0]:total_amount_box[2]])


# Print the extracted "Billed to" and "Supplied to" details
print("\n")
billed_to_text = billed_to_text.strip().split("\n")
for line in billed_to_text:
    if line.strip():
        print(line)

print("\n")
supplied_to_text = supplied_to_text.strip().split("\n")
for line in supplied_to_text:
    if line.strip():
        print(line)

# Print the extracted "Total Tax" and "Total Amount"
print("\n")
total_tax_text = total_tax_text.strip()
print(total_tax_text)

print("\n")
total_amount_text = total_amount_text.strip()
print(total_amount_text)

Invoice Number: 23331A0700000615
GSTIN: 33AABCI8842G1ZM
GSTIN/UIN: 33ACUPV0251A1ZS
State Code: 33
Invoice Date: 31-Jul-2023


Billed to
Name : TABTREE SOFTWARE SOLUTIONS
Address : ADDR
State / Place of Supply : Tamil Nadu
State code 233
GSTIN/ UIN : 83ACUPV0251A1ZS
ce Ril - 9022102000022631


Supplied to
Name : TABTREE SOFTWARE SOLUTIONS,
Address; ADDR
State : Tamil Nadu
State code : 33
GSTIN/ UIN : 33ACUPV0251A1ZS


Total Tax (B)

3.96


Total (A+B) —| 25.96
